In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')
snorkel_postgres = os.environ.get('SNORKELDB','').startswith('postgres')
print snorkel_postgres

False


### Parse

In [2]:
# If necessary:
import os
if snorkel_postgres:
    os.environ['SNORKELDBNAME'] = 'stg_temp_max'
    os.system("dropdb " + os.environ['SNORKELDBNAME'])
    os.system("createdb " + os.environ['SNORKELDBNAME'])
else:
    try:
        os.remove('snorkel.db')
    except:
        pass

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
import os

docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/html/'
pdf_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/pdf/'

if snorkel_postgres:
    from snorkel.async_parser import parse_corpus, HTMLParser, AsyncOmniParser
   
    # PARSE DEV
    print "Starting async parse..."
    doc_parser = HTMLParser()
    context_parser = AsyncOmniParser(blacklist=['style'], flatten=['span','br'], 
                                     tabular=True, lingual=True,
                                     visual=True, pdf_path=pdf_path)
    %time corpus = parse_corpus(session, 'Hardware Dev', docs_path,\
                                doc_parser, context_parser,\
                                max_docs=3, parallel=3)
    
else:
    from snorkel.parser import CorpusParser, HTMLParser, OmniParser
    from snorkel.utils import get_ORM_instance

    # PARSE DEV
    print "Starting sync parse..."
    doc_parser = HTMLParser(path=docs_path)
    context_parser = OmniParser(blacklist=['style'], flatten=['span','br'], 
                                tabular=True, lingual=True,
                                visual=True, pdf_path=pdf_path)
    cp = CorpusParser(doc_parser, context_parser, max_docs=125)
    %time corpus = cp.parse_corpus(name='Hardware Dev', session=session)

    session.add(corpus)
    session.commit()

print "%s contains %d documents" % (corpus, len(corpus))

In [ ]:
# If necessary:
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db snorkel.db\ corpus');

### Load

In [3]:
from snorkel.models import Corpus
import os

os.system('cp snorkel.db\ corpus snorkel.db');
from snorkel import SnorkelSession
session = SnorkelSession()

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Dev').one()
print "%s contains %d Documents" % (corpus, len(corpus))

# import cPickle as pickle
# pickle_file = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/parts_by_doc.pkl'
# with open(pickle_file, 'r') as f:
#     parts_by_doc = pickle.load(f)

Corpus (Hardware Dev) contains 122 Documents


### 1st pass

In [14]:
from hardware_utils import get_gold_parts_by_doc, get_manual_parts_by_doc
from snorkel.utils import get_ORM_instance
from snorkel.models import Corpus

corpus = get_ORM_instance(Corpus, session, 'Hardware Dev')

# parts_by_doc = get_gold_parts_by_doc()
parts_by_doc = get_manual_parts_by_doc(corpus)
# parts_by_doc = None

Finding part numbers...
[========================================] 100%
defaultdict(<type 'set'>, {u'BC818': set([u'25', u'40', u'16']), u'ONSEMICONDUCTOR_MMBT6521LT1': set([u'A']), u'VSMIS00373-1': set([u'-25', u'-40', u'-16']), u'UTCLS02155-1': set([u'25', u'40', u'16']), u'VISHS23888-1': set([u'A', u'C', u'B']), u'FAIRS25065-1': set([u'A', u'C', u'B']), u'FAIRCHILDSEMICONDUCTOR_KSC2310YTA': set([u'Y', u'R', u'O']), u'KECCS05435-1': set([u'A', u'C', u'B']), u'DISES00192-1': set([u'-25', u'-40', u'-16']), u'BC337': set([u'25', u'40', u'16']), u'BC546A_SERIES_B14-521026': set([u'A']), u'DISES00189-1': set([u'A', u'C', u'B']), u'VSMIS03793-1': set([u'25', u'40', u'16']), u'LITES00690-1': set([u'A', u'C', u'B']), u'MCCCS09741-1': set([u'25', u'40', u'16']), u'BC547': set([u'A', u'C', u'B']), u'BC546': set([u'A', u'C', u'B']), u'FAIRS19194-1': set([u'A', u'C', u'B']), u'LITES00689-1': set([u'40', u'16']), u'SANKEN_2SC4886': set([u'Y', u'O']), u'BC550': set([u'A', u'C', u'B']), u'DISES0064

In [15]:
import cPickle as pickle
pickle_file = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/parts_by_doc.pkl'

with open(pickle_file, 'w') as f:
    pickle.dump(parts_by_doc, f)

### Extraction

In [7]:
from hardware_utils import OmniNgramsPart, get_part_matcher, get_part_throttler_wrapper
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])
part_ngrams = OmniNgramsPart(n_max=3, parts_by_doc=parts_by_doc)
part_matcher = get_part_matcher()
part_throttler = get_part_throttler_wrapper()

In [8]:
# Attempt at alternative regex:
# from snorkel.matchers import RegexMatchSpan, Inverse, Intersect
# long_enough = RegexMatchSpan(rgx='[A-Z2][A-Z0-9\-]{4,}(?:\/(?:DG|HF))?', ignore_case=False)
# at_least_one_number = RegexMatchSpan(rgx='[A-Z0-9\-\/]*[0-9][A-Z0-9\-\/]*', ignore_case=False)
# at_least_one_letter = RegexMatchSpan(rgx='[A-Z0-9\-\/]*[A-Z][A-Z0-9\-\/]*', ignore_case=False) 
# bad_form = Inverse(RegexMatchSpan(rgx='(?:[A-Z]+\d)|(?:[A-Z0-9\-\/]{0,2}\-[A-Z0-9\-\/]*)'))
# bad_start = Inverse(RegexMatchSpan(rgx='(?:T[O0]|SOT|BRD|FIGURE|EHP)[A-Z0-9\-\/]*'))
# bad_end = Inverse(RegexMatchSpan(rgx='[A-Z0-9\-\/]*(TYP|MAX|MIN|BASE|HZ|MM|VDC|VDF)'))
# part_matcher = Intersect(long_enough, at_least_one_number, at_least_one_letter, 
#                          bad_form, bad_start, bad_end)

In [9]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, 
                        [part_ngrams], 
                        [part_matcher],
                        throttler=part_throttler)

In [10]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%
CPU times: user 5min 18s, sys: 1min 9s, total: 6min 27s
Wall time: 6min 33s
Candidate Set (Hardware Candidates) contains 3184 Candidates


### Evaluation

In [11]:
from hardware_utils import get_gold_dict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/hardware_dev_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
print "%d gold parts loaded" % len(gold_parts)

947 gold parts loaded


In [12]:
from hardware_utils import parts_f1

%time TP, FP, FN = parts_f1(candidates, gold_parts)

Scoring on Entity-Level Gold Data
Corpus Precision 0.888
Corpus Recall    0.718
Corpus F1        0.794
----------------------------------------
TP: 680 | FP: 86 | FN: 267

CPU times: user 14 s, sys: 372 ms, total: 14.4 s
Wall time: 14.5 s


In [ ]:
from pprint import pprint
pprint(FP)

In [16]:
%time TP, FP, FN = parts_f1(candidates, gold_parts, parts_by_doc)

Scoring on Entity-Level Gold Data
Corpus Precision 0.821
Corpus Recall    0.904
Corpus F1        0.861
----------------------------------------
TP: 856 | FP: 186 | FN: 91

CPU times: user 8.55 s, sys: 234 ms, total: 8.78 s
Wall time: 8.87 s


In [17]:
pprint(FP)

[(u'BC337', u'BC33716'),
 (u'BC337', u'BC33725'),
 (u'BC337', u'BC33740'),
 (u'BC337', u'BC33740BU'),
 (u'BC337', u'BC338-16'),
 (u'BC337', u'BC338-40'),
 (u'BC337', u'BC33825'),
 (u'BC337-D', u'BC337-XX'),
 (u'BC546', u'BC548A'),
 (u'BC546', u'BC549A'),
 (u'BC546', u'BC550A'),
 (u'BC546', u'BC550B'),
 (u'BC546-BC548C(TO-92)', u'BC546-BC548C'),
 (u'BC546A_SERIES_B14-521026', u'BC546BA'),
 (u'BC546A_SERIES_B14-521026', u'BC546CA'),
 (u'BC546A_SERIES_B14-521026', u'BC547BA'),
 (u'BC546A_SERIES_B14-521026', u'BC547CA'),
 (u'BC546A_SERIES_B14-521026', u'BC548BA'),
 (u'BC546A_SERIES_B14-521026', u'BC548CA'),
 (u'BC546A_SERIES_B14-521026', u'BC549BA'),
 (u'BC546A_SERIES_B14-521026', u'BC549CA'),
 (u'BC546A_SERIES_B14-521026', u'BC550BA'),
 (u'BC546A_SERIES_B14-521026', u'BC550CA'),
 (u'BC546_DIOTEC', u'BC546C'),
 (u'BC546_DIOTEC', u'BC549A'),
 (u'BC547', u'BC548A'),
 (u'BC547', u'BC548BU'),
 (u'BC547', u'BC549A'),
 (u'BC547', u'BC550A'),
 (u'BC547', u'BC550B'),
 (u'BC818-40LT1-D', u'BC818-40

In [ ]:
from hardware_utils import entity_to_candidates, get_part_throttler
from snorkel.lf_helpers import *

entity = FP[]
print entity
print

matches = entity_to_candidates(entity, candidates)
print "# Matches: %d" % len(matches)
print

In [ ]:
from hardware_utils import print_table_info

candidate = matches[1]
# print candidate
# print
print_table_info(candidate.part)
print list(get_col_ngrams(candidate.part))
# throttler = get_part_throttler()
# print throttler(candidate)

In [ ]:
# from snorkel.lf_helpers import *
# c = candidates[110]
# print c
# get_prev_sibling_tags(c.part).count('p')

The End.